In [ ]:
## datadam_gentrificastion_score

## 확인사항
## 1. KT 데이터 업데이트 여부
## 2. 건축물대장 데이터 업데이트 여부
## 3. IBK 제공 데이터 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
date = '202210'


# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 시군구코드
aws_cursor.execute(
"""
select distinct substring(pnu, 1, 5) as sig_cd
from m1.pnu
order by 1
"""
)
aws_conn.commit()
rows = aws_cursor.fetchall()
df = pd.DataFrame(rows, columns = ['sig_cd'])
lawd_cd = list(df['sig_cd'])

# INSERT
for i in lawd_cd:
    aws_cursor.execute(
    """
    insert into m2.datadam_gentrificastion_score (
    with date as (
        select '""" + date + """'::text as base_ym
    )
    select (select base_ym from date) as base_ym
         , A.pnu
         , substring(A.pnu, 1, 8) as emd_cd
         , case when 
                round(
                        (
                            ((avg(B.sale_12m) - avg(C.sale_12m_avg)) / avg(C.sale_12m_std) * 5 + 75) +
                            ((avg(B.comp_12m) - avg(C.comp_12m_avg)) / avg(C.comp_12m_std) * 5 + 75) +
                            ((avg(B.fltg_12m) - avg(C.fltg_12m_avg)) / avg(C.fltg_12m_std) * 5 + 75) +
                            ((avg(B.lpop_12m) - avg(C.lpop_12m_avg)) / avg(C.lpop_12m_std) * 5 + 75) +
                            ((avg(B.wpop_12m) - avg(C.wpop_12m_avg)) / avg(C.wpop_12m_std) * 5 + 75)
                        ) / 5, 2
                     ) > 100 then 100
                else
                round(
                        (
                            ((avg(B.sale_12m) - avg(C.sale_12m_avg)) / avg(C.sale_12m_std) * 5 + 75) +
                            ((avg(B.comp_12m) - avg(C.comp_12m_avg)) / avg(C.comp_12m_std) * 5 + 75) +
                            ((avg(B.fltg_12m) - avg(C.fltg_12m_avg)) / avg(C.fltg_12m_std) * 5 + 75) +
                            ((avg(B.lpop_12m) - avg(C.lpop_12m_avg)) / avg(C.lpop_12m_std) * 5 + 75) +
                            ((avg(B.wpop_12m) - avg(C.wpop_12m_avg)) / avg(C.wpop_12m_std) * 5 + 75)
                        ) / 5, 2
                     ) end as score
        , sysdate::date as create_at
        , sysdate::date as update_at
        , 'du.Park' as work_user
    from (
        -- 모집단 가져오기 : KT매출 & PNU존재 & 건물존재
        select T1.pnu
             , ST_Centroid(T2.poly::geometry) as poly
        from (
            -- KT매출 존재
            select distinct substring(bd_mgt_sn, 1, 19) as pnu
            from m1.kt_bldg_sales
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
                and substring(bd_mgt_sn, 1, 5) = '""" + str(i) + """' -- 시군구
        ) T1
        -- PNU 존재
        join m2.cremao_land T2
            on T1.pnu = T2.pnu
        -- 건물 존재
        join (
            select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
            from m1.bld_rgst br 
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
        ) T3
            on T1.pnu = T3.pnu
    ) A
    left join (
        -- 반경 150m 데이터 가져오기
        select ST_Centroid(T2.poly::geometry) as poly
             , T1.sale_12m, T1.comp_12m, T1.fltg_12m, T1.lpop_12m, T1.wpop_12m
        from (
            -- 증감률 데이터
            select *
            from m2.ibk_pnu_01 
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
                and sale_12m > 0 and comp_12m > 0 and fltg_12m > 0 and wpop_12m > 0 and lpop_12m > 0 -- 특이값 제외
                and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
        ) T1
        -- polygon
        join m2.cremao_land T2
            on T1.pnu = T2.pnu
    ) B
        on ST_DistanceSphere(A.poly, B.poly) < 150 -- 반경 150m PNU
    left join (
        -- 지역평균, 표준편차 가져오기
        select avg(sale_12m) as sale_12m_avg
             , avg(comp_12m) as comp_12m_avg
             , avg(fltg_12m) as fltg_12m_avg
             , avg(wpop_12m) as wpop_12m_avg
             , avg(lpop_12m) as lpop_12m_avg
             , stddev(sale_12m) as sale_12m_std
             , stddev(comp_12m) as comp_12m_std
             , stddev(fltg_12m) as fltg_12m_std
             , stddev(wpop_12m) as wpop_12m_std
             , stddev(lpop_12m) as lpop_12m_std
        from m2.ibk_pnu_01
        where 1=1
            and base_ym = (select base_ym from date) -- 기준월
            and sale_12m > 0 and comp_12m > 0 and fltg_12m > 0 and wpop_12m > 0 and lpop_12m > 0 -- 특이값 제외
            and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
    ) C
        on 1=1
    group by 1, 2, 3
    having score is not null -- score null 제외
    );
    """
    )
    aws_conn.commit()
    print(i, "Done")

11110 Done
11140 Done
11170 Done
11200 Done
11215 Done
11230 Done
11260 Done
11290 Done
11305 Done
11320 Done
11350 Done
11380 Done
11410 Done
11440 Done
11470 Done
11500 Done
11530 Done
11545 Done
11560 Done
11590 Done
11620 Done
11650 Done
11680 Done
11710 Done
11740 Done
26110 Done
26140 Done
26170 Done
26200 Done
26230 Done
26260 Done
26290 Done
26320 Done
26350 Done
26380 Done
26410 Done
26440 Done
26470 Done
26500 Done
26530 Done
26710 Done
27110 Done
27140 Done
